# Generation of a roof from a polygon


#### The libraries used are:
* *larlib,pyplasm and math*

At the beginning, four different regular polygons have been initialized.
There are also some simple support functions, such as:

* *mf(pl)*: takes as input a list of points and returns the point characterized by maximum coordinates;
* *iP()*: returns a list of the lines that describe the plan;
* *pE()*: describes the equation of the plane;
* *p2c()*: returns the midpoint of the segment data as input.


In [ ]:
from pyplasm import *
from larlib import *
import math

A = [3.0,3.0,0.0]
B = [3.0,16.0,0.0]
C = [10.0,16.0,0.0]
D = [10.0,3.0,0.0]

A1 = [10.0,8.0,0.0]
B1 = [10.0,16.0,0.0]
C1 = [20.0,16.0,0.0]
D1 = [20.0,8.0,0.0]

A = [5.0,5.0,0.0]
B = [5.0,20.0,0.0]
C = [20.0,20.0,0.0]
D = [20.0,5.0,0.0]
E = [15.0,5.0,0.0]
F = [15.0,15.0,0.0]
G = [10.0,15.0,0.0]
H = [10.0,5.0,0.0]

A = [4.0,4.0,0.0]
B = [4.0,20.0,0.0]
C = [20.0,20.0,0.0]
D = [20.0,4.0,0.0]
E = [13.0,4.0,0.0]
F = [13.0,8.0,0.0]
G = [16.0,8.0,0.0]
H = [16.0,15.0,0.0]
I = [8.0,15.0,0.0]
L = [8.0,8.0,0.0]
M = [11.0,8.0,0.0]
N = [11.0,4.0,0.0]

mg = 2

def mF(pL):
    upx = pL[0][0]
    upy = pL[0][1]
    for p in pL:
        if p[0]> upx : upx = p[0]
        if p[1]> upy : upy = p[1]
    return (upx,upy)

def iP(p1, p2, p3):
    A = numpy.array([p1[:3], p2[:3], p3[:3]])
    B = -1*numpy.array([p1[-1], p2[-1], p3[-1]])
    return p(list(numpy.linalg.solve(A,B)))

def pE(A,B,C):
    x1,y1,z1 = p(A)
    x2,y2,z2 = p(B)
    x3,y3,z3 = p(C) 
    v1 = [x2 - x1, y2 - y1, z2 - z1]
    v2 = [x3 - x1, y3 - y1, z3 - z1]
    vProduct = [v1[1] * v2[2] - v1[2] * v2[1], -1 * (v1[0] * v2[2] - v1[2] * v2[0]), v1[0] * v2[1] - v1[1] * v2[0]]
    a = vProduct[0]
    b = vProduct[1]
    c = vProduct[2]
    d = - (vProduct[0] * x1 + vProduct[1] * y1 + vProduct[2] * z1)
    return [a,b,c,d]

def p2c(pointA,pointB) :
    C = []
    m = max(pointA)
    for i=0 to m:
        C.append((pointA[i]+pointB[i])/2)
    return C

# Polygon

#### This is the function that creates the basic polygon. It takes as input a list of points. Examples of output:


![img](pol2.png)

![img](pol3.png)

![img](pol4.png)

![img](pol1.png)







In [ ]:
def generatePol(pL):
	segm = []
	segmL = []
	for i in range(0,len(pL)) :
        	if i == len(pL)-1 : 
            		segm.append(points2s(pL[i],pL[0]))
            		segmL.append([pL[i],pL[0]])
        	else: 
            		segm.append(points2s(pL[i],pL[i+1]))
            		segmL.append([pL[i],pL[i+1]])
	return (segm,segmL)

# Roof creator

#### The core of the program is the function *generateRoof*. Input:
* *list of points*: list of points initialized previously;
* *height*: height of the roof;
* *angle*: inclination of the roof pitches. 
#### The output is the model of the roof.
#### One of the support functions is getPointsTerrace(). It recognizes the critical points of the roof and orders them. Once that this step is done, the polygon representing the terrace is created. Finally, the roof is complete.


![img](3.png)

![img](4.png)

![img](1.png)

![img](2.png)



In [ ]:
def generateRoof (pointsList,height,angle):
    mg = 2
  
    xm = mF(pointsList)[0]
    ym = mF(pointsList)[1]

    ss = generatePol(pointsList)[0]
    ssList = generatePol(pointsList)[1]
    
    VIEW(COLOR(BLACK)(STRUCT(ss)))
    
    p = []
    for s in ssList :
        p.append(s2plane(s[0],s[1],angle))     
    p.append(COLOR(BLUE)(CUBOID([xm,ym,0])))  
    fsList = []
    for i in range(0,len(p)-1):
	if i != len(p)-mg : fsList.append(INTERSECTION([p[i],p[i+mg-1]]))
        else : fsList.append(INTERSECTION([p[i],p[0]]))


    fsL2 = []
    for f in fsList :
        fsL2.append(DIFFERENCE([f,STRUCT([T(3)(height),CUBOID([xm,ym,height*5])])]))
    
    """
    costruisco la lista degli effettivi piani inclinati del tetto facendo il JOIN a due a due 
    delle f precedentemente individuate
    """
    p2 = []
    for i in range (0,len(fsL2)) :
        if i == len(fsL2)-1 : p2.append(TEXTURE("tetto.jpg")(JOIN([fsL2[i],fsL2[0]])))
        else : p2.append(TEXTURE("tetto.jpg")(JOIN([fsL2[i],fsL2[i+1]])))
        
    
    roof = STRUCT(p2)
    roofUk = UKPOL(roof)

    pterrace = getPointsTerrace(pointsList,height,roofUk)

    terrace = STRUCT([T(3)(height),fillPolygon(pterrace)])  
    terrace = TEXTURE("terrazzo.jpg")(terrace)
    #VIEW(terrace)  
    
    roof = STRUCT([roof,terrace])
    return roof



def getPointsTerrace(pointsList,height,rk):
    maxps = []
    for p in rk[0] :
        if p[2] < height+0.1 and p[2] > height-0.1 :
            ins = False
            for eP in maxps :
                xf = p[0] > eP[0] + 0.1 or p[0] < eP[0] - 0.1 
                yf = p[1] > eP[1] + 0.1 or p[1] < eP[1] - 0.1
                if not xf and not yf : 
                    ins = True
            if not ins : 
                p[2] = 0.0
                maxps.append(p)

    maxps_orderer = []
    for point in pointsList :
        point2 = maxps[0]
        point2X = math.fabs(maxps[0][0]-point[0])
        point2Y = math.fabs(maxps[0][1]-point[1])
        distance = math.sqrt(point2X*point2X+point2Y*point2Y)
        for i in range (1,len(maxps)):
            p2roximityX = math.fabs(maxps[i][0]-point[0])
            p2roximityY = math.fabs(maxps[i][1]-point[1])
            newDistance = math.sqrt(p2roximityX*p2roximityX+p2roximityY*p2roximityY)
            if newDistance < distance :
                point2 = maxps[i]
                point2X = p2roximityX
                distance = newDistance
                
        maxps_orderer.append(point2)
        
    maxps = maxps_orderer
    return maxps

test1 = generateRoof(points,1.5,30)
VIEW(test1)


A = [3.0,3.0,0.0]
B = [3.0,16.0,0.0]
C = [10.0,16.0,0.0]
D = [10.0,3.0,0.0]

points = [A,B,C,D]

test1 = generateRoof(points,1.5,30)
VIEW(test1)

A1 = [10.0,8.0,0.0]
B1 = [10.0,16.0,0.0]
C1 = [20.0,16.0,0.0]
D1 = [20.0,8.0,0.0]


points2 = [A1,B1,C1,D1]

test2 = generateRoof(points2,1.5,30)
#VIEW(test2)
VIEW(STRUCT([test1,test2]))


A = [5.0,5.0,0.0]
B = [5.0,20.0,0.0]
C = [20.0,20.0,0.0]
D = [20.0,5.0,0.0]
E = [15.0,5.0,0.0]
F = [15.0,15.0,0.0]
G = [10.0,15.0,0.0]
H = [10.0,5.0,0.0]

points = [A,B,C,D,E,F,G,H]
test3 = generateRoof(points,1.8,60)
VIEW(test3)


A = [4.0,4.0,0.0]
B = [4.0,20.0,0.0]
C = [20.0,20.0,0.0]
D = [20.0,4.0,0.0]
E = [13.0,4.0,0.0]
F = [13.0,8.0,0.0]
G = [16.0,8.0,0.0]
H = [16.0,15.0,0.0]
I = [8.0,15.0,0.0]
L = [8.0,8.0,0.0]
M = [11.0,8.0,0.0]
N = [11.0,4.0,0.0]

points = [A,B,C,D,E,F,G,H,I,L,M,N]
#test3 = R([1,2])(PI)(generateRoof(points,1.8,60))
test4 = generateRoof(points,1.8,60)
VIEW(test4)
#VIEW(STRUCT([test3,test4]))


